In [1]:
!pip install currencyconverter

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

import numpy as np
import pandas as pd
import datetime

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

tf.random.set_seed(1)
np.random.seed(1)

import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import statistics
import plotly.graph_objects as go
from currency_converter import CurrencyConverter
from numpy.lib.stride_tricks import sliding_window_view

In [3]:
tf.keras.backend.set_image_data_format("channels_last")

In [4]:
role = get_execution_role()
prefix = 'sagemaker/encoder-decoder-trainer'
my_region = boto3.session.Session().region_name
my_region

'eu-west-1'

In [5]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='training-data-lstm', Key = 'keyword_level_complete_data2.csv')

In [6]:
dataset = pd.read_csv(obj['Body'])

In [7]:
dataset.drop(columns=["_id"], inplace=True)

In [8]:
dataset.sort_values(by=["sales_acc", "sales_camp"], inplace=True)
dataset.drop_duplicates(subset=["keywordId", "date"], keep='first', inplace=True)

In [9]:
# num_keywords_req = 20
# keyword_ids = dataset["keywordId"].unique()
# keywords_req = []
# keywordwise_sales = dataset.groupby("keywordId")["sales"].sum()
# nonzero_sales_keywords = keywordwise_sales[keywordwise_sales>=0]
# nonzero_sales_keywords.sort_values(ascending=False, inplace=True)
# keywords_req = list(nonzero_sales_keywords.index)
# keyword_ids = keywords_req[:num_keywords_req]
# dataset = dataset[dataset["keywordId"].isin(keyword_ids)]
# dataset.reset_index(inplace=True)

In [10]:
from datetime import date
import time
from tqdm import tqdm

def preprocessing_account_data(account_data):
    
    not_supported_currs = ["AED", "SAR"]
    manual_conversions = {"AED":0.27, "SAR":0.27}

    account_data["campaign_sales_perc"] = account_data["sales"]/account_data["sales_camp"]
    account_data["account_sales_perc"] = account_data["sales"]/account_data["sales_acc"]
    account_data["campaign_spend_perc"] = account_data["spend"]/account_data["spend_camp"]
    account_data["account_spend_perc"] = account_data["spend"]/account_data["spend_acc"]

    account_data.drop(columns=["sales_acc", "spend_acc", "sales_camp", "spend_camp"], inplace=True)

    c = CurrencyConverter(fallback_on_missing_rate=True)
    account_data["date"] = pd.to_datetime(account_data["date"])
    
    start = time.time()
    account_data["spend_usd"] = account_data.apply(lambda x: c.convert(x["spend"], x["currency_code"], "USD", date=x["date"]) \
                                                   if x["currency_code"] not in not_supported_currs else manual_conversions[x["currency_code"]]*x["spend"], axis=1)
    end = time.time()
    print(f"Spend Currency Conversion finished in: {end-start} seconds.")
    start = time.time()
    account_data["sales_usd"] = account_data.apply(lambda x: c.convert(x["sales"], x["currency_code"], "USD", date=x["date"]) \
                                                   if x["currency_code"] not in not_supported_currs else manual_conversions[x["currency_code"]]*x["sales"], axis=1)
    end = time.time()
    print(f"Sales Currency Conversion finished in: {end-start} seconds.")
    start = time.time()
    account_data["keyword_length"] = account_data.apply(lambda x: len(str(x["keywordText"])), axis=1)
    end = time.time()
    print(f"Keyword Length Calculation finished in: {end-start} seconds.")
    start = time.time()
    account_data["keyword_num_words"] = account_data.apply(lambda x: len(str(x["keywordText"]).split(" ")), axis=1)
    end = time.time()
    print(f"Keyword Num Words Calculation finished in: {end-start} seconds.")
    account_data.drop(columns=["accountId", "campaignId", "currency_code", "sales", "spend", "keywordText"], inplace=True)

    categorical_columns = ['matchType', 'country_code', 'campaign_type', 'targeting_type', 'budget_type', 'adFormat', 'tactic', 'costType']
    for column in categorical_columns:
        le = LabelEncoder()
        label_encoded_data = le.fit_transform(account_data[column])
        account_data.drop(columns=[column], inplace=True)
        account_data[column] = label_encoded_data

    account_data["cpc"] = account_data["spend_usd"]/account_data["clicks"]
    for keyword in tqdm(account_data["keywordId"].unique()):
      account_data.loc[account_data["keywordId"]==keyword]["cpc"] = account_data.loc[account_data["keywordId"]==keyword]["cpc"].interpolate("linear")
    account_data.fillna(0, inplace=True)
    account_data.drop(columns=["index"], inplace=True)

    account_data['year'] = account_data['date'].dt.year
    account_data['month'] = account_data['date'].dt.month
    account_data['day'] = account_data['date'].dt.day
    account_data['dayoftheweek'] = account_data['date'].dt.dayofweek

    return account_data

In [11]:
dataset = preprocessing_account_data(dataset)

Spend Currency Conversion finished in: 390.2006061077118 seconds.
Sales Currency Conversion finished in: 396.59042859077454 seconds.
Keyword Length Calculation finished in: 188.30184864997864 seconds.
Keyword Num Words Calculation finished in: 191.56805658340454 seconds.


100%|██████████| 790636/790636 [7:13:28<00:00, 30.40it/s]  


KeyError: "['index'] not found in axis"

In [13]:
dataset['year'] = dataset['date'].dt.year
dataset['month'] = dataset['date'].dt.month
dataset['day'] = dataset['date'].dt.day
dataset['dayoftheweek'] = dataset['date'].dt.dayofweek

In [15]:
from io import StringIO

bucket = 'training-data-lstm' # already created on S3
csv_buffer = StringIO()
dataset.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'processed_training_data.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'Q0D4RAY5H6R1356G',
  'HostId': 'ms7l5i0/GzVIFPkjPwfporzJyH435cOmr0QDhm3MvAbHv0QygPkQ7GpTYR58ERP5833OtgB/QcE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ms7l5i0/GzVIFPkjPwfporzJyH435cOmr0QDhm3MvAbHv0QygPkQ7GpTYR58ERP5833OtgB/QcE=',
   'x-amz-request-id': 'Q0D4RAY5H6R1356G',
   'date': 'Tue, 04 Jul 2023 02:57:39 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"64608341f3eefe515411f35a14ffd4ca"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"64608341f3eefe515411f35a14ffd4ca"',
 'ServerSideEncryption': 'AES256'}

In [16]:
HISTORICAL_DATA_WINDOW = 14
FUTURE_PREDICTION_WINDOW = 3

In [19]:
train_frames_X, train_frames_Y = [], []
test_frames_X, test_frames_Y = [], []
for keyword in tqdm(dataset["keywordId"].unique()):
    
  start = time.time()
  keyword_data = dataset.loc[dataset["keywordId"]==keyword]
  end = time.time()
  print("a: " + str(end-start))
  start = time.time()
  keyword_data.sort_values(by=["date"], inplace=True)
  end = time.time()
  print("b: " + str(end-start))
  start = time.time()
  spend_sales_scaler = preprocessing.MinMaxScaler()
  scaler = preprocessing.MinMaxScaler()
  keyword_data[["clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc", "year", "month", "day", "dayoftheweek"]] = scaler.fit_transform(keyword_data[["clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc", "year", "month", "day", "dayoftheweek"]])
  keyword_data[["spend_usd", "sales_usd"]] = spend_sales_scaler.fit_transform(keyword_data[["spend_usd", "sales_usd"]])
  end = time.time()
  print("c: " + str(end-start))
  start = time.time()
  train_size = (int)(len(keyword_data) * 0.7)
  train_data = keyword_data[:train_size]
  test_data = keyword_data[train_size-HISTORICAL_DATA_WINDOW-FUTURE_PREDICTION_WINDOW+1:]
  data_columns = keyword_data.columns
  train_data_arr, test_data_arr = np.array(train_data), np.array(test_data)

  if train_data_arr.shape[0] < HISTORICAL_DATA_WINDOW:
    continue

  train_data_frame_X = sliding_window_view(train_data_arr, window_shape = (HISTORICAL_DATA_WINDOW, train_data_arr.shape[1]))
  test_data_frame_X = sliding_window_view(test_data_arr, window_shape = (HISTORICAL_DATA_WINDOW, test_data_arr.shape[1]))
  print(train_data_frame_X.shape)
  train_data_frame_X = np.squeeze(train_data_frame_X)[:-FUTURE_PREDICTION_WINDOW]
  print(train_data_frame_X.shape)
  test_data_frame_X = np.squeeze(test_data_frame_X)[:-FUTURE_PREDICTION_WINDOW]
  train_data_frames_X, test_data_frames_X = train_data_frame_X.copy(), test_data_frame_X.copy()
  for frame in train_data_frames_X:
    train_frames_X.append(pd.DataFrame(frame, columns=data_columns))
  for frame in test_data_frames_X:
    test_frames_X.append(pd.DataFrame(frame, columns=data_columns))

  train_data_frame_Y = sliding_window_view(train_data_arr, window_shape = (FUTURE_PREDICTION_WINDOW, train_data_arr.shape[1]))
  test_data_frame_Y = sliding_window_view(test_data_arr, window_shape = (FUTURE_PREDICTION_WINDOW, test_data_arr.shape[1]))
  train_data_frame_Y = np.squeeze(train_data_frame_Y)[HISTORICAL_DATA_WINDOW:]
  test_data_frame_Y = np.squeeze(test_data_frame_Y)[HISTORICAL_DATA_WINDOW:]
  train_data_frames_Y, test_data_frames_Y = train_data_frame_Y.copy(), test_data_frame_Y.copy()
  for frame in train_data_frames_Y:
    train_frames_Y.append(pd.DataFrame(frame, columns=data_columns))
  for frame in test_data_frames_Y:
    test_frames_Y.append(pd.DataFrame(frame, columns=data_columns))
  end = time.time()
  print("d: " + str(end-start))
  break

  0%|          | 0/790636 [00:00<?, ?it/s]

a: 0.017305374145507812
b: 0.0007429122924804688
c: 0.007093191146850586
(166, 1, 14, 27)
(163, 14, 27)
d: 0.18839573860168457


In [16]:
train_frames_X[0]

,keywordId,date,clicks,impressions,orders,budget,campaign_sales_perc,account_sales_perc,campaign_spend_perc,account_spend_perc,...,targeting_type,budget_type,adFormat,tactic,costType,cpc,year,month,day,dayoftheweek
0,205964338670346,2022-01-24,0.090909,0.068207,0.0,4675.0,0.0,0.0,0.0,0.027825,...,1,0,1,0,0,0.053606,0.0,0.0,0.766667,0.0
1,205964338670346,2022-01-25,0.859091,0.60024,0.611111,4675.0,0.0,0.67148,0.0,0.354085,...,1,0,1,0,0,0.072317,0.0,0.0,0.8,0.166667
2,205964338670346,2022-01-26,0.75,0.532251,0.444444,4675.0,0.0,0.30954,0.0,0.315931,...,1,0,1,0,0,0.071489,0.0,0.0,0.833333,0.333333
3,205964338670346,2022-01-27,0.763636,0.542057,0.333333,4675.0,0.0,0.402265,0.0,0.36887,...,1,0,1,0,0,0.080321,0.0,0.0,0.866667,0.5
4,205964338670346,2022-01-28,0.777273,0.617128,0.444444,4675.0,0.0,0.445964,0.0,0.432424,...,1,0,1,0,0,0.087222,0.0,0.0,0.9,0.666667
5,205964338670346,2022-01-29,0.818182,0.57093,0.5,4675.0,0.0,0.574664,0.0,0.404257,...,1,0,1,0,0,0.078162,0.0,0.0,0.933333,0.833333
6,205964338670346,2022-01-30,0.695455,0.606014,0.111111,4675.0,0.0,0.115632,0.0,0.384731,...,1,0,1,0,0,0.091735,0.0,0.0,0.966667,1.0
7,205964338670346,2022-01-31,0.745455,0.631401,0.388889,4675.0,0.0,0.310904,0.0,0.350578,...,1,0,1,0,0,0.083865,0.0,0.0,1.0,0.0
8,205964338670346,2022-02-01,0.654545,0.590869,0.277778,4675.0,0.0,0.422649,0.0,0.375574,...,1,0,1,0,0,0.100033,0.0,0.090909,0.0,0.166667
9,205964338670346,2022-02-02,0.668182,0.515254,0.333333,4675.0,0.0,0.418551,0.0,0.420803,...,1,0,1,0,0,0.102785,0.0,0.090909,0.033333,0.333333


In [17]:
train_frames_embed, test_frames_embed = [], []
embedding_columns = ["keyword_length", "keyword_num_words", "budget", "matchType", "country_code", "campaign_type", "targeting_type", "budget_type", "adFormat", "tactic", "costType"]

for i in range(len(train_frames_X)):
  train_frames_embed.append(train_frames_X[i].loc[0][embedding_columns])
  train_frames_X[i].drop(columns=embedding_columns, inplace=True)

for i in range(len(test_frames_X)):
  test_frames_embed.append(test_frames_X[i].loc[0][embedding_columns])
  test_frames_X[i].drop(columns=embedding_columns, inplace=True)

In [18]:
for i in range(len(train_frames_X)):
  train_frames_X[i].drop(columns=["keywordId", "date"], inplace=True)
  train_frames_Y[i].drop(columns=embedding_columns, inplace=True)
  train_frames_Y[i].drop(columns=["keywordId", "date", "year", "month", "day", "dayoftheweek", "clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc"], inplace=True)

for i in range(len(test_frames_X)):
  test_frames_X[i].drop(columns=["keywordId", "date"], inplace=True)
  test_frames_Y[i].drop(columns=embedding_columns, inplace=True)
  test_frames_Y[i].drop(columns=["keywordId", "date", "year", "month", "day", "dayoftheweek", "clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc"], inplace=True)

In [19]:
train_frames_decoder_input, test_frames_decoder_input = [], []

for i in range(len(train_frames_Y)):
  train_frames_decoder_input.append(train_frames_Y[i]["cpc"])
  train_frames_Y[i].drop(columns=["cpc"], inplace=True)

for i in range(len(test_frames_Y)):
  test_frames_decoder_input.append(test_frames_Y[i]["cpc"])
  test_frames_Y[i].drop(columns=["cpc"], inplace=True)

In [20]:
print("Example Input to Encoder:")
print(train_frames_X[0])
print("Example Output of Decoder:")
print(train_frames_Y[0])
print("Example Embedding Input:")
print(train_frames_embed[0])
print("Example Input to Decoder:")
print(train_frames_decoder_input[0])

Example Input to Encoder:
      clicks impressions    orders campaign_sales_perc account_sales_perc  \
0   0.090909    0.068207       0.0                 0.0                0.0   
1   0.859091     0.60024  0.611111                 0.0            0.67148   
2       0.75    0.532251  0.444444                 0.0            0.30954   
3   0.763636    0.542057  0.333333                 0.0           0.402265   
4   0.777273    0.617128  0.444444                 0.0           0.445964   
5   0.818182     0.57093       0.5                 0.0           0.574664   
6   0.695455    0.606014  0.111111                 0.0           0.115632   
7   0.745455    0.631401  0.388889                 0.0           0.310904   
8   0.654545    0.590869  0.277778                 0.0           0.422649   
9   0.668182    0.515254  0.333333                 0.0           0.418551   
10  0.690909    0.531924  0.333333                 0.0           0.384065   
11  0.645455    0.483766  0.166667                

In [21]:
NUM_EMBEDDING_FEATURES = train_frames_embed[0].shape[0]
NUM_HISTORICAL_FEATURES = train_frames_X[0].shape[1]

In [22]:
encoder_input = keras.Input(shape=(HISTORICAL_DATA_WINDOW, NUM_HISTORICAL_FEATURES))
encoder_lstm1 = layers.LSTM(32, return_sequences=True)(encoder_input)
encoder_lstm1 = layers.ReLU()(encoder_lstm1)
batch_norm1 = layers.BatchNormalization()(encoder_lstm1)
encoder_lstm2 = layers.LSTM(64, return_sequences=True)(batch_norm1)
encoder_lstm2 = layers.ReLU()(encoder_lstm2)
batch_norm2 = layers.BatchNormalization()(encoder_lstm2)
encoder_output = layers.LSTM(32)(batch_norm2)
encoder_output = layers.ReLU()(encoder_output)

embedding_input = keras.Input(shape=(NUM_EMBEDDING_FEATURES, ))
embedding_layer1 = layers.Dense(32)(embedding_input)
embedding_layer1 = layers.ReLU()(embedding_layer1)
batch_norm3 = layers.BatchNormalization()(embedding_layer1)
embedding_layer2 = layers.Dense(64)(batch_norm3)
embedding_layer2 = layers.ReLU()(embedding_layer2)
batch_norm4 = layers.BatchNormalization()(embedding_layer2)
embedding_output = layers.Dense(32)(batch_norm4)
embedding_output = layers.ReLU()(embedding_output)

embedding_encoder_concatenate = layers.Concatenate()([encoder_output, embedding_output])
embedding_encoder_concatenate = layers.Dense(32)(embedding_encoder_concatenate)
embedding_encoder_concatenate = layers.ReLU()(embedding_encoder_concatenate)
future_cpc = keras.Input(shape=(FUTURE_PREDICTION_WINDOW, ))

decoder_input = layers.RepeatVector(FUTURE_PREDICTION_WINDOW)(embedding_encoder_concatenate)
future_cpc = layers.Reshape((-1, 1))(future_cpc)
decoder_input = layers.Concatenate()([decoder_input, future_cpc])
decoder_lstm1 = layers.LSTM(32, return_sequences=True)(decoder_input)
decoder_lstm1 = layers.ReLU()(decoder_lstm1)
batch_norm5 = layers.BatchNormalization()(decoder_lstm1)
decoder_lstm2 = layers.LSTM(16, return_sequences=True)(batch_norm5)
decoder_lstm2 = layers.ReLU()(decoder_lstm2)
batch_norm6 = layers.BatchNormalization()(decoder_lstm2)
decoder_output = layers.LSTM(2, return_sequences=True)(batch_norm6)
decoder_output_fin = layers.ReLU()(decoder_output)

# Create the model
ED_lstm_model = tf.keras.Model(inputs=[encoder_input, embedding_input, future_cpc], outputs=decoder_output_fin)
ED_lstm_model.compile(optimizer="adam", loss='mean_squared_error')

2023-06-29 12:27:42.916600: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [23]:
ED_lstm_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14, 14)]     0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 14, 32)       6016        ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 32)           384         ['input_2[0][0]']                
                                                                                              

In [24]:
train_frames_X, train_frames_Y = np.array(train_frames_X).astype(np.float32), np.array(train_frames_Y).astype(np.float32)
train_frames_embed, train_frames_decoder_input = np.array(train_frames_embed).astype(np.float32), np.array(train_frames_decoder_input).astype(np.float32)

test_frames_X, test_frames_Y = np.array(test_frames_X).astype(np.float32), np.array(test_frames_Y).astype(np.float32)
test_frames_embed, test_frames_decoder_input = np.array(test_frames_embed).astype(np.float32), np.array(test_frames_decoder_input).astype(np.float32)

In [25]:
train_frames_X, train_frames_Y = np.nan_to_num(train_frames_X).astype(np.float32), np.nan_to_num(train_frames_Y).astype(np.float32)
train_frames_embed, train_frames_decoder_input = np.nan_to_num(train_frames_embed).astype(np.float32), np.nan_to_num(train_frames_decoder_input).astype(np.float32)

test_frames_X, test_frames_Y = np.nan_to_num(test_frames_X).astype(np.float32), np.nan_to_num(test_frames_Y).astype(np.float32)
test_frames_embed, test_frames_decoder_input = np.nan_to_num(test_frames_embed).astype(np.float32), np.nan_to_num(test_frames_decoder_input).astype(np.float32)

In [26]:
print(train_frames_X.shape)
print(train_frames_Y.shape)
print(train_frames_embed.shape)
print(train_frames_decoder_input.shape)

(4154, 14, 14)
(4154, 3, 2)
(4154, 11)
(4154, 3)


In [35]:
EPOCHS = 20
BATCH_SIZE = 32
model_path = "./trained_model.ckpt"

history = ED_lstm_model.fit(
    [train_frames_X, train_frames_embed, train_frames_decoder_input],  # Pass input tensors as a tuple
    train_frames_Y,
    batch_size=BATCH_SIZE,
    steps_per_epoch=len(train_frames_X)//BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=([test_frames_X, test_frames_embed, test_frames_decoder_input], test_frames_Y),  # Pass validation data as a tuple
    validation_steps=len(test_frames_X)//BATCH_SIZE,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0,
            patience=10,
            verbose=1,
            mode='min'
        ),
        tf.keras.callbacks.ModelCheckpoint(
            model_path,
            monitor='val_loss',
            save_best_only=True,
            mode='min',
            verbose=1
        )
    ]
)

Epoch 1/15
127/129 [============================>.] - ETA: 0s - loss: 0.0449
Epoch 1: val_loss improved from inf to 0.04892, saving model to ./trained_model.ckpt


INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


129/129 [==============================] - 47s 251ms/step - loss: 0.0450 - val_loss: 0.0489
Epoch 2/15
127/129 [============================>.] - ETA: 0s - loss: 0.0261
Epoch 2: val_loss improved from 0.04892 to 0.04693, saving model to ./trained_model.ckpt


INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


...

INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


129/129 [==============================] - 30s 233ms/step - loss: 0.0141 - val_loss: 0.0216
Epoch 15/15
128/129 [============================>.] - ETA: 0s - loss: 0.0139
Epoch 15: val_loss improved from 0.02162 to 0.02074, saving model to ./trained_model.ckpt


INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


INFO:tensorflow:Assets written to: ./trained_model.ckpt/assets


129/129 [==============================] - 30s 232ms/step - loss: 0.0139 - val_loss: 0.0207


In [28]:
model_path = "./trained_model.ckpt"
ED_lstm_model.load_weights(model_path)

In [29]:
def plot(predicted_frames_Y, test_frames_Y, date_frames, keyword):

  sales_fig = go.Figure()
  spend_fig = go.Figure()
  date_frames = np.array(date_frames)

  for i in range(FUTURE_PREDICTION_WINDOW):

    predicted_sales = predicted_frames_Y[:, i, 1]
    predicted_spend = predicted_frames_Y[:, i, 0]

    actual_sales = test_frames_Y[:, i, 1]
    actual_spend = test_frames_Y[:, i, 0]

    dates = date_frames[:, i]

    sales_fig.add_trace(go.Scatter(x=dates, y=predicted_sales, mode='markers+lines', name=f'Predicted Sales - {i} step'))
    spend_fig.add_trace(go.Scatter(x=dates, y=predicted_spend, mode='markers+lines', name=f'Predicted Spend - {i} step'))

    sales_fig.add_trace(go.Scatter(x=dates, y=actual_sales, mode='markers+lines', name=f'Actual Sales - {i} step'))
    spend_fig.add_trace(go.Scatter(x=dates, y=actual_spend, mode='markers+lines', name=f'Actual Spend - {i} step'))

  sales_fig.update_layout(title=f"Sales Prediction - {keyword}")
  spend_fig.update_layout(title=f"Spend Prediction - {keyword}")

  sales_fig.show()
  spend_fig.show()

  return sales_fig, spend_fig

from sklearn.metrics import mean_squared_error

def calc_rmse(yhat, y):
    return mean_squared_error(y, yhat, squared=False)

def calc_pearson_correlation(yhat, y):
    return np.corrcoef(y, yhat)[0, 1]

def analyze(predicted_frames_Y, test_frames_Y, date_frames, keyword):

  sales_fig, spend_fig = plot(predicted_frames_Y, test_frames_Y, date_frames, keyword)
  spend_rmse = calc_rmse(predicted_frames_Y[:, 0, 0], test_frames_Y[:, 0, 0])
  sales_rmse = calc_rmse(predicted_frames_Y[:, 0, 1], test_frames_Y[:, 0, 1])
  spend_pearson_coeff = calc_pearson_correlation(predicted_frames_Y[:, 0, 0], test_frames_Y[:, 0, 0])
  sales_pearson_coeff = calc_pearson_correlation(predicted_frames_Y[:, 0, 1], test_frames_Y[:, 0, 1])

  return sales_fig, spend_fig, spend_rmse, sales_rmse, spend_pearson_coeff, sales_pearson_coeff

In [54]:
def create_cpc_bid_landscape(model, test_frames_x, test_frames_embed, test_frames_decoder_input, date_frames, scaler, min_cpc, max_cpc):
    cpc_steps = np.linspace(min_cpc, max_cpc, 30)
    predicted_frames_cpc = []
    date_frames = np.squeeze(np.array(date_frames)[:, 0])
    for cpc in cpc_steps:
        decoder_input = np.full(test_frames_decoder_input.shape, cpc)
        predicted_frames = model.predict([test_frames_x, test_frames_embed, decoder_input])
        predicted_frames_cpc += [scaler.inverse_transform(np.squeeze(predicted_frames[:, 0, :]))]
    predicted_frames_cpc = np.array(predicted_frames_cpc)
    spend_cpc_landscape = np.squeeze(predicted_frames_cpc[:, :, 0])
    sales_cpc_landscape = np.squeeze(predicted_frames_cpc[:, :, 1])
    fig = go.Figure(data=[go.Surface(z=spend_cpc_landscape, x=cpc_steps, y=date_frames)])
    fig.show()
    return predicted_frames_cpc

In [55]:
import os
embedding_columns = ["keyword_length", "keyword_num_words", "budget", "matchType", "country_code", "campaign_type", "targeting_type", "budget_type", "adFormat", "tactic", "costType"]
idx = 0

test_results_path = "./test_results/"
results_df = pd.DataFrame(columns=["keywordId", "spend_rmse", "sales_rmse", "spend_corr", "sales_corr"])

for keyword in dataset["keywordId"].unique():

  test_frames_X, test_frames_Y = [], []
  test_frames_embed, test_frames_decoder_input = [], []

  keyword_data = dataset.loc[dataset["keywordId"]==keyword]
  keyword_data.sort_values(by=["date"], inplace=True)
  spend_sales_scaler = preprocessing.MinMaxScaler()
  scaler = preprocessing.MinMaxScaler()
  keyword_data[["clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc", "year", "month", "day", "dayoftheweek"]] = scaler.fit_transform(keyword_data[["clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc", "year", "month", "day", "dayoftheweek"]])
  keyword_data[["spend_usd", "sales_usd"]] = spend_sales_scaler.fit_transform(keyword_data[["spend_usd", "sales_usd"]])
  train_size = (int)(len(keyword_data) * 0.8)
  #test_data = keyword_data[:train_size]
  test_data = keyword_data[train_size-HISTORICAL_DATA_WINDOW-FUTURE_PREDICTION_WINDOW+1:]
  data_columns = keyword_data.columns
  test_data_arr = np.array(test_data)

  test_data_frame_X = sliding_window_view(test_data_arr, window_shape = (HISTORICAL_DATA_WINDOW, test_data_arr.shape[1]))
  test_data_frame_X = np.squeeze(test_data_frame_X)[:-FUTURE_PREDICTION_WINDOW]
  test_data_frames_X = test_data_frame_X.copy()

  for frame in test_data_frames_X:
    test_frames_X.append(pd.DataFrame(frame, columns=data_columns))

  test_data_frame_Y = sliding_window_view(test_data_arr, window_shape = (FUTURE_PREDICTION_WINDOW, test_data_arr.shape[1]))
  test_data_frame_Y = np.squeeze(test_data_frame_Y)[HISTORICAL_DATA_WINDOW:]
  test_data_frames_Y = test_data_frame_Y.copy()
  for frame in test_data_frames_Y:
    test_frames_Y.append(pd.DataFrame(frame, columns=data_columns))

  date_frames = []

  for i in range(len(test_frames_X)):
    test_frames_embed.append(test_frames_X[i].loc[0][embedding_columns])
    test_frames_X[i].drop(columns=embedding_columns, inplace=True)
    date_frames.append(test_frames_Y[i]["date"])
    test_frames_X[i].drop(columns=["keywordId", "date"], inplace=True)
    test_frames_Y[i].drop(columns=embedding_columns, inplace=True)
    test_frames_Y[i].drop(columns=["keywordId", "date", "year", "month", "day", "dayoftheweek", "clicks", "impressions", "orders", "campaign_sales_perc", "campaign_spend_perc", "account_sales_perc", "account_spend_perc"], inplace=True)

  for i in range(len(test_frames_Y)):
    test_frames_decoder_input.append(test_frames_Y[i]["cpc"])
    test_frames_Y[i].drop(columns=["cpc"], inplace=True)

  test_frames_X, test_frames_Y = np.array(test_frames_X).astype(np.float32), np.array(test_frames_Y).astype(np.float32)
  test_frames_embed, test_frames_decoder_input = np.array(test_frames_embed).astype(np.float32), np.array(test_frames_decoder_input).astype(np.float32)

  test_frames_X, test_frames_Y = np.nan_to_num(test_frames_X).astype(np.float32), np.nan_to_num(test_frames_Y).astype(np.float32)
  test_frames_embed, test_frames_decoder_input = np.nan_to_num(test_frames_embed).astype(np.float32), np.nan_to_num(test_frames_decoder_input).astype(np.float32)
    
  predicted_frames_cpc = create_cpc_bid_landscape(ED_lstm_model, test_frames_X, test_frames_embed, test_frames_decoder_input, date_frames, spend_sales_scaler, 0.1, 2)
  print(predicted_frames_cpc)
  predicted_frames_cpc = np.array(predicted_frames_cpc)
  print(predicted_frames_cpc.shape)
  break
#   predicted_frames_Y = ED_lstm_model.predict([test_frames_X, test_frames_embed, test_frames_decoder_input])

#   for i in range(len(predicted_frames_Y)):
#     predicted_frames_Y[i] = spend_sales_scaler.inverse_transform(predicted_frames_Y[i])
#     test_frames_Y[i] = spend_sales_scaler.inverse_transform(test_frames_Y[i])
#   sales_fig, spend_fig, spend_rmse, sales_rmse, spend_pearson_coeff, sales_pearson_coeff = analyze(predicted_frames_Y, test_frames_Y, date_frames, keyword)
#   if not os.path.exists(os.path.join(test_results_path, str(keyword)+"/")):
#     os.makedirs(os.path.join(test_results_path, str(keyword)+"/"))
#   sales_fig.write_html(os.path.join(test_results_path, str(keyword)+"/sales_fig.html"))
#   spend_fig.write_html(os.path.join(test_results_path, str(keyword)+"/spend_fig.html"))
#   results_df.loc[len(results_df)] = [keyword, spend_rmse, sales_rmse, spend_pearson_coeff, sales_pearson_coeff]
    
#   create_cpc_bid_landscape()
#   idx += 1
#   if idx==3:
#     break

# results_df.to_csv("./test_results_df.csv")

3/3 [==============================] - 0s 7ms/step


[[[13.059706  79.43084  ]
  [10.31967   64.69512  ]
  [10.9504175 71.500275 ]
  ...
  [ 8.560711  55.322403 ]
  [ 8.236521  53.826283 ]
  [ 8.725404  55.547947 ]]

 [[16.786104  87.16278  ]
  [13.26769   70.535126 ]
  [14.049337  76.420334 ]
  ...
  [12.152802  64.52919  ]
  [11.735417  62.921516 ]
  [12.3192625 64.24289  ]]

 [[20.347122  95.586685 ]
  [16.19313   73.67868  ]
  [17.042175  77.481094 ]
  ...
  [15.755266  69.857254 ]
  [15.310029  68.38916  ]
  [15.856124  68.92801  ]]

 ...

 [[23.501385  92.429054 ]
  [22.957533  81.32476  ]
  [23.395445  80.6011   ]
  ...
  [21.333117  69.8842   ]
  [21.218481  72.09985  ]
  [21.424889  72.203804 ]]

 [[23.316984  93.023834 ]
  [22.775427  81.804596 ]
  [23.225668  80.96149  ]
  ...
  [20.90252   69.63919  ]
  [20.794802  71.71094  ]
  [20.998018  71.84289  ]]

 [[23.143057  93.66237  ]
  [22.604416  82.339455 ]
  [23.06561   81.37802  ]
  ...
  [20.442032  70.22348  ]
  [20.367994  71.42369  ]
  [20.567501  71.58205  ]]]
(30, 73, 2